In [1]:
from IPython.core.magic import register_cell_magic
import importlib

@register_cell_magic
def spark_class(line, cell):
    module = line.strip()
    f = "{0}.py".format(module)

    with open(f, "w") as fw:
        fw.write(cell)

    globals()[module] = importlib.import_module(module)
    sc.addPyFile(f)

In [2]:
%%spark_class veggie

class Vegetable(object):
    veg_type=None
    def __init__(self):
        self.is_raw=True
        self.peels=True
        
class Carrot(Vegetable):
    veg_type="carrot"
    
class Potato(Vegetable):
    veg_type="potato"

class Leek(Vegetable):
    veg_type="leek"
    
class PieceOfVeggie(Vegetable):
    def __init__(self, origin_veggie=None):
        self.origin=origin_veggie
        self.is_raw=origin_veggie.is_raw
        self.peels=origin_veggie.peels
        self.veg_type=origin_veggie.veg_type
        
    @staticmethod
    def from_veggie(veggie):
        return PieceOfVeggie(origin_veggie=veggie)
    
class MashedVeggie(object):
    def __init__(self, veggie_list):
        self.veggie_list=veggie_list
    @staticmethod
    def from_veggies(veggie_list):
        return MashedVeggie(veggie_list=veggie_list)
    def __add__(self, other):
        return MashedVeggie.from_veggies(veggie_list=self.veggie_list+other.veggie_list)
    
class Soup(object):
    def __init__(self, mashed_veggie):
        self.mashed_veggie = mashed_veggie
    @staticmethod
    def from_mashed_veggies(mash_veggie):
        return Soup(mashed_veggie=mash_veggie)


class Boil(object):
    def __init__(self, time):
        self.time=time
        
    def __call__(self, veggies):
        for veggie in veggies:
            veggie.is_raw=False
        return veggies

class Peel(object):
    def __call__(self, veggie):
        veggie.peels=False
        return veggie

class Cut(object):
    def __init__(self, n):
        self.n=n
    def __call__(self, veggie):
        return [PieceOfVeggie.from_veggie(veggie)]*self.n    

    
class Blend(object):
    def __call__(self, veggies):
        return MashedVeggie(veggies)    


In [33]:
class Vegetable(object):
    veg_type=None
    def __init__(self):
        self.is_raw=True
        self.peels=True
        
class Carrot(Vegetable):
    veg_type="carrot"
    
class Potato(Vegetable):
    veg_type="potato"

class Leek(Vegetable):
    veg_type="leek"
    
class PieceOfVeggie(Vegetable):
    def __init__(self, origin_veggie=None):
        self.origin=origin_veggie
        self.is_raw=origin_veggie.is_raw
        self.peels=origin_veggie.peels
        self.veg_type=origin_veggie.veg_type
        
    @staticmethod
    def from_veggie(veggie):
        return PieceOfVeggie(origin_veggie=veggie)
    
class MashedVeggie(object):
    def __init__(self, veggie_list):
        self.veggie_list=veggie_list
    @staticmethod
    def from_veggies(veggie_list):
        return MashedVeggie(veggie_list=veggie_list)
    def __add__(self, other):
        return MashedVeggie.from_veggies(veggie_list=self.veggie_list+other.veggie_list)
    
class Soup(object):
    def __init__(self, mashed_veggie):
        self.mashed_veggie = mashed_veggie
    @staticmethod
    def from_mashed_veggies(mash_veggie):
        return Soup(mashed_veggie=mash_veggie)


class Boil(object):
    def __init__(self, time):
        self.time=time
        
    def __call__(self, veggies):
        for veggie in veggies:
            veggie.is_raw=False
        return veggies

class Peel(object):
    def __call__(self, veggie):
        veggie.peels=False
        return veggie

class Cut(object):
    def __init__(self, n):
        self.n=n
    def __call__(self, veggie):
        return [PieceOfVeggie.from_veggie(veggie)]*self.n    

    
class Blend(object):
    def __call__(self, veggies):
        return MashedVeggie(veggies)    


In [35]:
import random
ingredients = sc.parallelize(range(18)).map(lambda i: random.choice([veggie.Carrot(), veggie.Potato(), veggie.Leek()]))

def custom_boil(veggie_list):
    
    time_to_boil_table = {
        'carrot': 25,
        'potato': 20,
        'leek': 15
    }
    
    return veggie.Boil(
        time_to_boil_table[veggie_list[0].veg_type]
    )(veggie_list)


ingredients = ingredients\
                    .map(lambda i: veggie.Peel()(i))\
                    .flatMap(lambda i: veggie.Cut(10)(i)) \
                    .map(lambda veggie: (veggie.veg_type, [veggie]))\
                    .reduceByKey(lambda x,y:x+y)\
                    .map(lambda x: x[1])\
                    .map(custom_boil)\
                    .map(lambda i: veggie.Blend()(i))\
                    .reduce(lambda x, y: x+y)

Soup.from_mashed_veggies(ingredients)